# Задача 1 Удвоение чисел и получение первого результата

In [1]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

data = [
    [175, 790, 103, 479, 433, 406, 510, 377, 184, 777, 990, 321],
    [466, 538, 210, 881, 873, 411, 856, 965, 537, 243, 749, 229, 545,
     312, 651, 342, 328, 152, 473, 479, 257, 899, 362, 584, 888],
    [393, 488, 955, 771, 536, 922, 182, 564, 581, 998, 556, 813, 482, 940, 947, 267, 721, 944, 820, 993],
    [691, 532, 354, 831, 241, 724, 547, 580, 191, 742, 263, 149, 661, 804, 819, 246, 519, 698],
    [373, 235, 860, 591, 592, 227, 455, 113, 426, 181, 741, 723, 998, 667, 827],
    [924, 880, 704, 133, 538, 795, 364, 687, 775, 925, 445, 659, 883, 620, 391, 
     836, 779, 617, 528, 914, 424],
    [324, 643, 171, 208, 330, 306, 559, 927, 871, 284, 438, 644, 447, 893, 287],
    [993, 788, 192, 169, 549, 162, 324, 213, 277, 376, 391, 243, 749, 229, 545, 516, 260, 798],
    [497, 971, 765, 137, 543, 498, 583, 649, 558, 488, 882, 907, 589, 151, 724, 
     689, 134, 492, 124, 114, 147, 734, 524,
     658, 441, 908, 192, 240],
    [575, 873, 922, 950, 375, 555, 351, 582, 659, 629, 619, 851, 661, 804, 819, 
     246, 519, 698, 286, 639, 593, 773, 157],
]

def process_number(number):
    """Обработка одно числа из списка"""
    result = number * 2
    time.sleep(0.02)
    return result

def process_list(numbers):
    """Обработка списка чисео"""
    result = 0
    for number in numbers:
        result += process_number(number)
    return result

with ThreadPoolExecutor(max_workers=10) as executor:
    # Создание и запуск задач
    futures = [executor.submit(process_list, l) for l in data]

    # Получение результатов задач
    for future in as_completed(futures):
        first_list_sum = future.result()
        print(f"Сумма чисел в первом обработанном списке: {first_list_sum}")
        break

Сумма чисел в первом обработанном списке: 11090


# Задача 2 Поиск и суммирование чисел через цепочку файлов

In [2]:
import pathlib
import zipfile
from multiprocessing import Pool
import platform

zip_with_filenames_path = pathlib.Path.cwd().joinpath('data/path_8_8.zip')
data_zip_path = pathlib.Path.cwd().joinpath('data/recursive_challenge_8_8.zip')

In [3]:
def prepared_filename(filename):
    """Преобразование стиль пути к файлу Linux <-> Windows"""
    if platform.system()=='Linux':
        filename = filename.replace('\\','/')    
    else:
        filename = filename.replace('/', '\\')    
    return filename

def process_file(filename, zip_with_filenames_path, data_zip_path):
    """Обработать один файл"""
    # откроем архив с файлами-ссылками
    zip_with_filenames = zipfile.ZipFile(zip_with_filenames_path)
    # считаем ссылку на файл с данными
    data_file_path = prepared_filename(zip_with_filenames.read(filename).decode())
    # откроем архив с данными
    data_zip = zipfile.ZipFile(data_zip_path)
    # считаем число из data_file_path
    num = int(data_zip.read(data_file_path).decode())
    return num


In [4]:
# Получение списка файлов
paths_file = zipfile.ZipFile(zip_with_filenames_path)
files = [(p.filename, zip_with_filenames_path, data_zip_path) for p in paths_file.infolist() if not p.is_dir()]

In [5]:
# Однопоточная обработка файлов (для сравнения)
total_sum = 0
for f in files:
    total_sum += process_file(f[0], f[1], f[2])
print(f'Итоговая сумма: {total_sum}')

Итоговая сумма: 5152208


In [6]:
# Многопроцессорная обработка файлов
total_sum = 0
with Pool(processes=12) as pool:
    for r in pool.starmap(process_file, files):
        total_sum += r
print(f'Итоговая сумма: {total_sum}')

Итоговая сумма: 5152208
